In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import sklearn 
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.preprocessing import OneHotEncoder
import math

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Dataloader

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# Custom Dataset class
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data.float()

    def __getitem__(self, index):
        return self.data[index].float()

    def __len__(self):
        return len(self.data)

In [4]:
def train_epoch(train_loader, model, optimizer, loss_function):
    losses = []
    # get a batch of training data from the train_loader (DataLoader obj)
    for i, data in enumerate(train_loader):
        x = data[:,:-1].to(device)
        t = data[:,-1].to(device)
        # make predictions for this batch
        y = model(x)
        # Compute the loss
        loss = loss_function(y, t)
        # zero out the gradients so that it will not accumulate through each iteration
        optimizer.zero_grad()
        # Compute the gradents with the backward call (backprop)
        loss.backward()
        # Update weight using gradient descent 
        optimizer.step()
        losses.append(loss.item())
    return np.mean(losses)

def eval_epoch(valid_loader, model, loss_function):
    rmses = []
    with torch.no_grad(): 
        for i, data in enumerate(valid_loader):
            x = data[:,:-1].to(device)
            t = data[:,-1].to(device)
            # Compute prediction
            y = model(x).squeeze()
            
            rmse = math.sqrt(loss_function(y,t))
            rmses.append(rmse)
            
    return np.mean(rmses)

# Define Model Architectures

In [19]:
# Define the neural network architecture
leaky_relu = nn.LeakyReLU(negative_slope=0.2)
class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(69, 64)
#         self.fc2 = nn.Linear(64, 48)
#         self.fc3 = nn.Linear(48, 32)
#         self.fc4 = nn.Linear(32,32)
#         self.fc5 = nn.Linear(32, 16)  
#         self.fc6 = nn.Linear(16, 1)   

#     def forward(self, x):
#         x = leaky_relu(self.fc1(x))
#         x = leaky_relu(self.fc2(x))
#         x = leaky_relu(self.fc3(x))     
#         x = leaky_relu(self.fc4(x))   
#         x = leaky_relu(self.fc5(x))
#         x = self.fc6(x)
#         return x
    
    def __init__(self):
        super(Net, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(69, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))     
        x = self.fc4(x)
        return x

# Create an instance of the neural network
model = Net()
model = model.to(device)
criterion = nn.MSELoss()

# Load in Data

In [34]:
df_np = np.load('processed_train_set.npy')
df_np.shape

(1710670, 70)

In [35]:
# load in np array and DRAW SAMPLE
num_samples = int(df_np.shape[0]*0.1)

num_rows = df_np.shape[0]
sampled_indices = np.random.choice(num_rows, size=num_samples, replace=False)

In [36]:
# convert np to tensor
df = torch.tensor(df_np[sampled_indices])

In [37]:
df.shape

torch.Size([171067, 70])

In [38]:
# Create a sample dataset
dataset = MyDataset(df)

# Define the sizes of train, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset)-train_size

In [39]:
# Split the dataset
batch_size = 32
train_data, val_data = random_split(dataset, [train_size, val_size])

# Create DataLoader objects for train, validation, and test sets
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(val_data, batch_size=batch_size)
# test_loader = DataLoader(test_data, batch_size=32)

# Train Model

In [40]:
def train_model(model = model, model_name = 'main_model', epochs = 10, lr=0.00001, batch_size = batch_size):
    # Define the optimizer
    train_rmses = []
    valid_rmses = []
#     optimizer = torch.optim.Adam(model.parameters(), lr)
    optimizer = torch.optim.Adam(model.parameters())
    epochs = 10
    train_accs, valid_accs = [], []
    for epoch in range(epochs):

        model.train() # gradient tracking is on

        train_loss = train_epoch(train_loader, model, optimizer, criterion)

        model.eval() # we don't need gradients on to do reporting

        train_rmse = eval_epoch(train_loader, model, criterion)
        valid_rmse = eval_epoch(valid_loader, model, criterion)
        
#         train_rmses.append(train_rmse)
#         valid_rmses.append(valid_rmse)

        print(f"Epoch: {epoch+1}, Train Loss: {train_loss:>0.4f}, Train mse: {train_rmse:>0.2f},\
              Validation Accuracy: {valid_rmse:>0.2f} \n")
    torch.save(model.state_dict(), './{}'.format(model_name))


In [41]:
%%time
train_model(model = model, model_name = 'main_model', \
epochs = 10, lr=0.00001, batch_size = batch_size)

/home/zsiyed/.local/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([21])) that is different to the input size (torch.Size([21, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1, Train Loss: 459573.1677, Train mse: 549.61,              Validation Accuracy: 553.52 

Epoch: 2, Train Loss: 459554.9581, Train mse: 552.13,              Validation Accuracy: 556.37 

Epoch: 3, Train Loss: 459568.2017, Train mse: 551.99,              Validation Accuracy: 555.02 

Epoch: 4, Train Loss: 459499.7235, Train mse: 550.66,              Validation Accuracy: 553.28 

Epoch: 5, Train Loss: 459523.0944, Train mse: 550.01,              Validation Accuracy: 553.76 

Epoch: 6, Train Loss: 459659.3452, Train mse: 552.47,              Validation Accuracy: 556.07 

Epoch: 7, Train Loss: 459575.8223, Train mse: 548.58,              Validation Accuracy: 552.20 

Epoch: 8, Train Loss: 459549.2536, Train mse: 548.27,              Validation Accuracy: 552.07 

Epoch: 9, Train Loss: 459562.1353, Train mse: 550.50,              Validation Accuracy: 554.60 

Epoch: 10, Train Loss: 459455.4579, Train mse: 549.81,              Validation Accuracy: 552.36 

CPU times: user 2min 10s, sys

In [ ]:
# saved_model = Net()
# saved_model.load_state_dict(torch.load('./cse151B_project'))
# saved_model

# Test Model (unfinished cuz eric poopoo code)

In [ ]:
test = pd.read_csv('test_public.csv')
test.head()

In [ ]:
# encode time
# Convert Unix timestamp to datetime
test['datetime'] = pd.to_datetime(test['TIMESTAMP'], unit='s')

# Extract year, month, day of the week, and hour of the day
test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day_of_week'] = test['datetime'].dt.dayofweek
test['hour_of_day'] = test['datetime'].dt.hour
test_datetime = test

# Create an instance of the OneHotEncoder
encoder = OneHotEncoder()
# categorical_cols = ['CALL_TYPE', 'year', 'month', 'day_of_week', 'hour_of_day']
categorical_cols = ['CALL_TYPE']

test_df_hot = test_datetime[categorical_cols].values

# Fit the encoder on the categorical variable
encoder.fit(test_df_hot)

# Transform the categorical variable into one-hot encoded features
test_hot_encoded = encoder.transform(test_df_hot).toarray()
test_hot_encoded = pd.DataFrame(test_hot_encoded)
# print(test_hot_encoded.shape)

test_df_OHE = pd.concat([test_datetime,test_hot_encoded], axis = 1).drop(['TIMESTAMP','TRIP_ID','CALL_TYPE', 'DAY_TYPE', 'datetime'], axis = 1) # basic
test_df_OHE = test_df_OHE.fillna(0)
# test_hot = pd.DataFrame(test_hot_encoded, columns = encoder.get_feature_names_out(['CALL_TYPE', 'year', 'month', 'day_of_week', 'hour_of_day']))

#basic
test_df_basic = test_df_OHE.copy()
test_df_basic['MISSING_DATA'] = test_df_basic['MISSING_DATA'].astype('int')
test_df_basic.shape
#basic

# test_hot = test_hot.reindex(columns=df_hot.columns, fill_value=0)
# test_hot
cols = ['ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID', 'MISSING_DATA', 'year', 'month', 'day_of_week', 'hour_of_day', 'CALL_TYPE_A', 'CALL_TYPE_B', 'CALL_TYPE_C']
test_df_basic = test_df_basic.reindex(columns=cols, fill_value=0)
test_df_basic.head()

In [ ]:
test_tensor = torch.tensor(test_df_basic.values).float().to(device)
test_tensor.shape

In [ ]:
test_pred = []
for i in range(test_tensor.shape[0]):
    test_pred.append(model_basic(test_tensor[i]).item())
test_pred

In [ ]:
test_data = {'TRIP_ID': test['TRIP_ID'],
    'TRAVEL_TIME': xg_pred}
output = pd.DataFrame(data = test_data)
output

In [ ]:
output.to_csv('zeds_may25_test.csv', index=False)
